In [ ]:
!pip install langchain_community
!pip install replicate
!pip install requests
!pip install beautifulsoup4

In [2]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata

# Set the API token
api_token = userdata.get('REPLICATE_KEY')
os.environ["REPLICATE_API_TOKEN"] = api_token

In [15]:
parameters = {
  "top_k": 5,
  "top_p": 1.0,
  "max_tokens": 4096,
  "min_tokens": 0,
  "random_seed": None,
  "temperature": 0,
  "repetition_penalty": 1.0,
  "stopping_criteria": "length (256 tokens)",
  "stopping_sequence": None
}

llm = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    model_kwargs=parameters
)

In [5]:
import requests
from bs4 import BeautifulSoup
import re

def hapus_spasi_lebih(text):
    if not isinstance(text, str) or not text.strip():
        return ""

    return re.sub(r'\s+', ' ', text).strip()

def extract_berita_detik(soup):
    container = soup.find(id="detikdetailtext")
    if not container:
        return ""

    paragraphs = container.find_all('p')
    combined = " ".join(p.get_text(strip=True) for p in paragraphs)
    return hapus_spasi_lebih(combined)

def ambil_data(url):
  response = requests.get(url)
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    judul = hapus_spasi_lebih(soup.find('h1').text)
    konten = extract_berita_detik(soup)
    return f"""
    Judul berita: {judul}

    Konten Berita: {konten}
    """
  else:
    return "Url Berita Tidak Valid"

In [29]:
url = "https://www.detik.com/edu/sekolah/d-7959673/4-temuan-kemendikdasmen-di-spmb-2025-jual-beli-kursi-pemalsuan-dokumen"
berita = ambil_data(url)

prompt = f"""
Kamu adalah asisten analisis berita.
Tugasmu adalah menganalisis dan meringkas satu berita online dalam bahasa Indonesia, dengan hasil akhir yang rapi, jelas, dan mudah dipahami oleh masyarakat umum.
Fokuskan pada semua poin penting yang disebutkan dalam berita, terutama bila berita menyebutkan daftar temuan, indikasi, atau poin-poin utama.

Lakukan dua hal:
1. Klasifikasikan berita berdasarkan elemen berikut:
- Kategori Utama: (Politik, Ekonomi, Hukum, Sosial, Pendidikan, Olahraga, Kesehatan, Teknologi, Lingkungan, Bencana, Hiburan, dll.)
- Tokoh Utama (jika ada): (misalnya: Ahmad, Sukron)
- Lokasi Kejadian (jika ada): (misalnya: Jakarta, Papua)
- Waktu Kejadian: (misalnya: 20 Juli 2025)
- Sentimen berita: (Positif / Negatif / Netral)
- Isu Khusus (jika relevan): (misalnya: Korupsi, Pendidikan anak, Lingkungan hidup, dll.)
- Keaslian berita: (misalnya: Hoaks, Fakta, jika hoaks berikan alasannya)

2. Ringkasan (maksimal 3 - 7 kalimat)
- Buat ringkasan yang sederhana, tidak teknis, dan bisa dipahami masyarakat awam.
- Sebutkan apa yang terjadi dan siapa yang terlibat.
- Jika berita menyebutkan beberapa temuan atau daftar poin, sebutkan semua poin tersebut secara singkat.
- Jangan hanya menulis ulang lead berita — benar-benar ambil inti isinya.
- Jangan lebih dari 7 kalimat.

Berikut beritanya
{berita}
"""

output = llm.invoke(prompt)

print(output)

1. Klasifikasi berita:
- Kategori Utama: Politik (karena terkait dengan instansi pemerintah dan proses pemilihan)
- Tokoh Utama: Kemendikbud, SPMB (Badan Pemilihan Bantuan dan Pendidikan)
- Lokasi Kejadian: Netral (tidak menyebutkan lokasi spesifik)
- Waktu Kejadian: 20 Juli 2025 (walaupun tidak jelas apakah ini waktu publikasi berita atau waktu kejadian)
- Sentimen berita: Negatif (karena berita mengungkap kasus pemalsuan dokumen)
- Isu Khusus: Korupsi, keberlanjutan proses pemilihan
- Keaslian berita: Fakta (tidak ada alasan menunjukkan hoaks)

2. Ringkasan:
Kemendikbud, badan pemerintah yang bertanggung jawab untuk pendidikan, terdakwa melakukan pemalsuan dokumen selama proses pemilihan SPMB 2025. Berita menyebutkan empat (4) temuan yang menunjukkan kasus pemalsuan, salah identitas, dan penipuan. Ini menciptakan sentimen negatif karena menunjukkan potensi korupsi dalam proses pemilihan dan menyebabkan kebimbangan tentang keberlanjutan proses tersebut. Berita juga menyebutkan beberap